In [2]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import plotly.plotly as ply
import plotly.graph_objs as go
from datetime import datetime
import numpy as np

In [10]:
mol_2018 = pd.read_csv(r'/home/ale/Documents/TP FINAL/Mol 2018.csv', sep=",")

mol_2018.columns = map(str.upper, mol_2018.columns)
mol_2018['FECHA'] = pd.to_datetime(mol_2018['FECHA'])
mol_2018['Month'] = pd.DatetimeIndex(mol_2018['FECHA']).month
mol_2018['Year'] = pd.DatetimeIndex(mol_2018['FECHA']).year

mol_2018.drop(['PAX_PAGOS'],axis=1,inplace=True)
mol_2018.drop(['PAX_PASES_PAGOS'],axis=1,inplace=True)
mol_2018.drop(['PAX_FRANQ'],axis=1,inplace=True)

mol_2018.dropna(how='any', inplace=True)
mol_2018['TOTAL'].fillna(0, inplace=True)
mol_2018=mol_2018[mol_2018['MOLINETE'].apply(lambda x: "LINEA_D" in x )]

mol_2018.to_csv("Linea_D_sintwitt.csv", sep=';', encoding='utf-8')

,PERIODO,FECHA,DESDE,HASTA,LINEA,MOLINETE,ESTACION,TOTAL,Month,Year
0,201801,2018-01-01,08:00:00,08:15:00,LINEA_A,LINEA_A_CBARROS_S_TURN01,CASTRO BARROS,1.0,1,2018
1,201801,2018-01-01,08:00:00,08:15:00,LINEA_A,LINEA_A_LIMA_S_TURN03,LIMA,4.0,1,2018
2,201801,2018-01-01,08:00:00,08:15:00,LINEA_A,LINEA_A_PASCO_TURN01,PASCO,1.0,1,2018
3,201801,2018-01-01,08:00:00,08:15:00,LINEA_A,LINEA_A_PERU_S_TURN01,PERU,4.0,1,2018
4,201801,2018-01-01,08:00:00,08:15:00,LINEA_A,LINEA_A_PJUNTA_S_TURN02,PRIMERA JUNTA,2.0,1,2018


In [ ]:
#mol_2018 = pd.read_csv(r'/home/ale/Documents/TP FINAL/Mol 2018.csv', sep=",")

#mol_2018.head()

#fig, ax = plt.subplots(figsize=(50,5))
#ax.plot_date(mol_2018['FECHA'], mol_2018['TOTAL'], color='r')
#plt.show()
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go


from plotly import tools

import pandas as pd
from datetime import datetime


    
#https://plot.ly/python/figure-labels/
#https://plot.ly/python/offline/
data1 = go.Scatter(x=mol_2018.DESDE,y=mol_2018['TOTAL'],mode ='lines+markers',
                   name="Total horario",
                   text="")


data = [go.Scatter(x=mol_2018.DESDE, y=mol_2018['TOTAL'],mode ='lines+markers',
                   name="Total horario",
                   text="")]

py.iplot(data)



/home/ale/anaconda3/lib/python3.7/site-packages/plotly/plotly/plotly.py:248: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.

